# Gradient-based solver for ridge regression

In this notebook, you will create a **gradient descent** solver for **ridge regression** and then compare it to the built-in solver in `sklearn.linear_model`.

## 1. Set up notebook and create data set

After loading in some standard packages, we create a synthetic data set consisting of data points `(x,y)`:
* `x`: 100-dimensional vector whose coordinates are independent draws from a standard normal (Gaussian) distribution
* `y`: response value given by `y = wx + e` where `w` is a target regression function and `e` is Gaussian noise

We will fix `w` to be the 100-dimensional vector whose first ten coordinates are exactly 1.0, and whose remaining coordinates are zero. Thus only the first ten coordinates of `x` are relevant to the regression task.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)

The following procedure, **generate_data**, creates a data set of a specified number of points. It is invoked as follows:
* `trainx, trainy = generate_data(n)`

Here:
* `n` is the target number of points
* `trainx`: `nx100` array of data points
* `trainy`: array of `n` response values

In [2]:
def generate_data(n):
    d = 100
    w = np.zeros(d)
    for i in range(0,10):
        w[i] = 1.0
    #
    trainx = np.random.normal(size=(n,d))
    e = np.random.normal(size=(n))
    trainy = np.dot(trainx, w) + e
    #
    return trainx, trainy

In [3]:
trainx, trainy=generate_data(200)

In [4]:
len(trainx[0,:])

100

## 2. Gradient descent solver for ridge regression

<font color="magenta">**For you to do:**</font> Define a procedure, **ridge_regression_GD**, that uses gradient descent to solve the ridge regression problem. It is invoked as follows:

* `w,b,losses = ridge_regression_GD(x,y,C)`

Here, the input consists of:
* training data `x,y`, where `x` and `y` are numpy arrays of dimension `n`-by-`d` and `n`, respectively (if there are `n` training points)
* regularization constant `C`

The function should find the `d`-dimensional vector `w` and offset `b` that minimize the ridge regression loss function (with regularization constant `C`), and return:
* `w` and `b`
* `losses`, an array containing the ridge regression loss at each iteration

<font color="magenta">Advice:</font> First figure out the derivative, which has a relatively simple form. Next, when implementing gradient descent, think carefully about two issues.

1. What is the step size?
2. When has the procedure converged?

Take the time to experiment with different ways of handling these.

In [26]:
def ridge_regression_GD(x,y,C):
    
    one_vector=np.ones((len(x),1)) # Making a vector made of 1s
    
    x_tilda=np.append(x, one_vector, 1) 
    # Assimilate the one_vector into the data matrix (for the 'b' intercept term).
    # The data matrix in [n x (d+1)] dim. 
    
    step = 0.1 # An arbitrarily chosen step 
    
    threshold = 0.01 # The while-loop stopper. 
    # If the length of 'del_L' is less then this threshold,
    # then stop the while loop and return the final w_tilta.
    
    del_L_length = 1 
    # The initial length of 'del_L' to run the first while loop.
    
    w_tilda = np.zeros((len(x_tilda[0,:]),1)) 
    # The initial model w_tilta in (d+1) dim. All of the elements are 0
    
    losses = []
    while del_L_length > threshold: #while loop 
    
        del_L = np.zeros((len(x_tilda[0,:]),1))
        
        for j in range(len(x_tilda[0,:])): # the number of data features: d + 1
            in_sum=np.zeros((len(x_tilda),1))
            in_sum_for_losses = np.zeros((len(x_tilda),1))
            wj = w_tilda[j]
            
            for i in range(len(in_sum)): # the number of data points: n
                in_sum[i,0]=(y[i]-np.dot(x_tilda[i,:],w_tilda))*x_tilda[i,j]
                in_sum_for_losses[i,0]=(y[i]-np.dot(x_tilda[i,:],w_tilda))**2
            summation_term = np.sum(in_sum)
            summation_term_for_losses = np.sum(in_sum_for_losses)
            addition_term = 2*C*wj
                             
            del_L[j,0] = -2*summation_term + addition_term                                       
            # UPDATE del_L
                             
        w_tilda = w_tilda - step * del_L
        # UPDATE w_tilda
        del_L_length = np.dot(del_L.transpose(), del_L)
        
        losses.append(summation_term_for_losses+C*np.dot(w_tilda.transpose(), w_tilda))
                             
    w=w_tilda[0:-1]
    b=w_tilda[-1]                         
    #losses=1 #Update this              
    return w,b,losses

In [ ]:
#This function is NOT working now... 
# w is diverging WHY? 
# check the sign and maybe treat b separately

In [ ]:
#TEST

In [84]:
n = 200
x,y = generate_data(n)
C = 1.0

one_vector=np.ones((len(x),1)) # Making a vector made of 1s
x_tilda=np.append(x, one_vector, 1) 
step = 0.1

del_L_length = 1 
iteration = 1

w_tilda = np.zeros((len(x_tilda[0,:]),1)) 
del_L = np.zeros((len(x_tilda[0,:]),1))
losses = []
for n in range(iteration):

    for j in range(len(x_tilda[0,:])): # the number of data features: d + 1
        in_sum=np.zeros((len(x_tilda),1))
        in_sum_for_losses = np.zeros((len(x_tilda),1))
        wj = w_tilda[j]
    
        for i in range(len(in_sum)): # the number of data points: n
            in_sum[i]=(y[i]-np.dot(x_tilda[i,:],w_tilda))*x_tilda[i,j]
            in_sum_for_losses[i,0]=(y[i]-np.dot(x_tilda[i,:],w_tilda))**2
        summation_term = np.sum(in_sum)
        summation_term_for_losses = np.sum(in_sum_for_losses)
        addition_term = 2*C*wj
    
        del_L[j,0] = -2*summation_term + addition_term
    
    w_tilda = w_tilda - step * del_L
    del_L_length = np.dot(del_L.transpose(), del_L)
    losses.append(summation_term_for_losses+C*np.dot(w_tilda.transpose(), w_tilda))

                             
w=w_tilda[0:-1]
b=w_tilda[-1]

In [85]:
w_tilda

array([[ 3.42720280e+01],
       [ 4.33798348e+01],
       [ 2.49180011e+01],
       [ 2.25210120e+01],
       [ 4.36614033e+01],
       [ 2.89425688e+01],
       [ 3.74106245e+01],
       [ 4.09037755e+01],
       [ 4.30929246e+01],
       [ 4.22351617e+01],
       [ 6.36209349e+00],
       [-6.17494946e+00],
       [-1.32739929e+00],
       [-6.28829381e-01],
       [-1.92656064e+00],
       [-1.16691345e+01],
       [ 1.24331086e+01],
       [-2.71150873e+00],
       [ 9.72857527e+00],
       [ 1.09437471e+00],
       [-1.02370606e+01],
       [ 1.89042334e+00],
       [-2.65666041e+00],
       [ 3.20789933e+00],
       [-2.04444211e+00],
       [-3.02240058e+00],
       [ 3.78762799e-02],
       [ 3.71468816e+00],
       [ 5.68590895e+00],
       [-5.66980682e+00],
       [-5.30927912e+00],
       [ 5.19708690e+00],
       [ 1.10440912e+01],
       [ 3.11195686e+00],
       [ 2.68996603e+00],
       [-8.90847644e+00],
       [ 5.99598203e+00],
       [ 1.41374781e+01],
       [ 4.5

Let's try it out and print a graph of the loss values during the optimization process.

In [27]:
# Generate 200 data points
n = 200
x,y = generate_data(n)
# Set regularization constant
C = 1.0
# Run gradient descent solver
w, b, losses,del_L = ridge_regression_GD(x,y,C)
# Plot the losses

# plt.plot(losses,'r')
# plt.xlabel('Iterations', fontsize=14)
# plt.ylabel('Loss', fontsize=14)
# plt.show()

<font color="magenta">**Something to think about**</font>

1. In setting the step size, does it work to use a fixed schedule 1/t? Why or why not?

2. Can you set up the gradient descent procedure in such a way that on each iteration, the loss monotonically decreases?


## 3. Evaluate the gradient descent solver

Now let's compare the regressor found by your gradient descent procedure to that returned by the built-in ridge regression solver in `sklearn`. We will compare them in two ways:
* Their MSE values
* The distance between the corresponding `w`-vectors

The latter should be smaller than 10^{-4}.


In [ ]:
def compute_mse(w,b,x,y):
    residuals = y - (np.dot(x, w) + b)
    return np.dot(residuals, residuals)/n

In [ ]:
# Generate 200 data points
n = 200
x,y = generate_data(n)
# Set regularization constant
C = 10.0
# Run gradient descent solver and compute its MSE
w, b, losses = ridge_regression_GD(x,y,C)
# Use built-in routine for ridge regression and compute MSE
regr = linear_model.Ridge(alpha=C)
regr.fit(x, y)
# Print MSE values and L2 distance between the regression functions
print("MSE of gradient descent solver: ", compute_mse(w,b,x,y))
print("MSE of built-in solver: ", mean_squared_error(regr.predict(x), y))
print("Distance between w-coefficients: ", np.linalg.norm(w-regr.coef_))

<font color="magenta">**Something to think about**</font>

The data was originally generated using a linear function in which only ten of the 100 features (the first ten) were relevant. Does the vector `w` returned by ridge regression correctly identify the relevant features?